In [71]:
import numpy as np
import pandas as pd
import re

In [72]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [73]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
print(stopwords.words('english')) #they don't add much weigtage to our datasets

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [75]:
df=pd.read_csv("/content/drive/MyDrive/Projects_Drishti/Fake_News_predictor/train.csv")

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
df.shape

(20800, 5)

In [78]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [79]:
df.isnull().sum() #we have enough data set to train our model, no need of preprocessing


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [80]:
df=df.fillna('') #replacing missing values with null strings

In [81]:
df['content']=df['title']+' '+df['author']

In [82]:
x=df.drop(columns='label',axis=1)
y=df['label']
print(x)
print(y)

          id  ...                                            content
0          0  ...  House Dem Aide: We Didn’t Even See Comey’s Let...
1          1  ...  FLYNN: Hillary Clinton, Big Woman on Campus - ...
2          2  ...  Why the Truth Might Get You Fired Consortiumne...
3          3  ...  15 Civilians Killed In Single US Airstrike Hav...
4          4  ...  Iranian woman jailed for fictional unpublished...
...      ...  ...                                                ...
20795  20795  ...  Rapper T.I.: Trump a ’Poster Child For White S...
20796  20796  ...  N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797  20797  ...  Macy’s Is Said to Receive Takeover Approach by...
20798  20798  ...  NATO, Russia To Hold Parallel Exercises In Bal...
20799  20799  ...            What Keeps the F-35 Alive David Swanson

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

In [83]:
#do Stemming ie reducing a word to its root word ie removing all prefix and suffixes
#eg for actor,actress,acting --> act(reducing to accurate root word)
port_stem=PorterStemmer()
def stemming(content):
    stem_cont=re.sub('[^a-zA-Z]',' ',content) #removes unwanted numbers and commas from data(they would be replaced by as space)
    stem_cont=stem_cont.lower()
    #converting all the alphabets to lower case
    stem_cont=stem_cont.split()
    print(stem_cont)
    print("\n")
    stem_cont=[port_stem.stem(word) for word in stem_cont if not word in stopwords.words('english')] #removing the stopwords
    print(stem_cont)
    stem_cont=' '.join(stem_cont)
    return stem_cont
    

In [84]:
df['content']=df['content'].apply(stemming)

Streaming output truncated to the last 5000 lines.
['dennis', 'jackson']


['denni', 'jackson']
['jane', 'fonda', 'i', 'won', 't', 'call', 'trump', 'by', 'his', 'name', 'i', 'call', 'him', 'the', 'predator', 'in', 'chief', 'breitbart', 'ian', 'hanchett']


['jane', 'fonda', 'call', 'trump', 'name', 'call', 'predat', 'chief', 'breitbart', 'ian', 'hanchett']
['not', 'just', 'hillary', 'entire', 'obama', 'administration', 'exposed', 'for', 'using', 'private', 'email', 'to', 'avoid', 'foia', 'requests', 'jack', 'burns']


['hillari', 'entir', 'obama', 'administr', 'expos', 'use', 'privat', 'email', 'avoid', 'foia', 'request', 'jack', 'burn']
['the', 'libertarian', 'party', 'just', 'gave', 'up', 'on', 'winning', 'tacitly', 'endorsed', 'hillary', 'over', 'trump', 'natalie', 'dickinson']


['libertarian', 'parti', 'gave', 'win', 'tacitli', 'endors', 'hillari', 'trump', 'natali', 'dickinson']
['democratic', 'panic', 'matthew', 'vadum']


['democrat', 'panic', 'matthew', 'vadum']
['get', 'ready

In [85]:
print(df['content'])

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: content, Length: 20800, dtype: object


In [86]:
X=df['content'].values
Y=df['label'].values

In [87]:
print(type(X)) #1->fake news and 0->real news

<class 'numpy.ndarray'>


In [88]:
vect=TfidfVectorizer() #it counts the number of times a word is repeating in a paragraph and then tells its importance by its frequency
#Tf stands for term frequency and idf stands for inverse doc frequency
#word repeated several times loses meaning and hence termed as fake news
X=vect.fit_transform(X)


In [89]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=4) #stratify is used to spread fake and real news in equal proportion
logreg=LogisticRegression()

In [90]:
logreg.fit(x_train,y_train)

LogisticRegression()

In [91]:
from sklearn.metrics import accuracy_score
pred=logreg.predict(x_test)
acc=accuracy_score(pred,y_test)
print(acc)

0.9742788461538462


In [92]:
x_new=x_test[5]
pred=logreg.predict(x_new)
print(pred)
if(pred==0):
    print("real")
else:
    print("fake")

[0]
real


In [93]:
#predicting accuracy from naive bias theorem

In [94]:
from sklearn import naive_bayes
naiveBias=naive_bayes.MultinomialNB()

In [95]:
naiveBias.fit(x_train,y_train)

MultinomialNB()

In [96]:
from sklearn.metrics import accuracy_score

In [103]:
pred2=naiveBias.predict_proba(x_test)[:,1]
print(pred2)

[0.10983253 0.78150045 0.84322534 ... 0.96805901 0.00332263 0.13281917]


In [108]:
print(accuracy_score(y_test,pred2))

0.9555288461538461


In [105]:
for i in  range(len(pred2)):
  if(pred2[i]>=0.5):
    pred2[i]=1
  else:
    pred2[i]=0
    

In [106]:
accur2=accuracy_score(y_test,pred2)

In [107]:
print(accur2)

0.9555288461538461


In [95]:
##hence we get to see that logistic regression worked better